In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
data = pd.read_csv('train.csv')
train = data.sample(frac=0.7, random_state=1)
test = data[data.index.isin(train.index)==False]
#train['Obs']=range(1,805)  # maybe change index from 1 to 804
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,RL,NaN,6853,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2009,WD,220000
791,160,FV,30.0,3180,Pave,Pave,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,151000
571,20,RL,60.0,6960,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,Shed,500,11,2009,WD,120500
224,20,RL,70.0,8050,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2007,WD,127000
498,20,RL,NaN,9556,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,8,2007,WD,160000


In [2]:
rows, columns = train.shape
print(rows)
print(columns)

563
79


In [3]:
train.describe().round(2) # without object type

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,563.00,451.00,563.00,563.00,563.00,563.00,563.00,560.00,563.00,563.00,...,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00
mean,56.86,68.29,9800.64,5.97,5.63,1970.00,1982.61,103.04,439.10,54.17,...,90.55,41.44,25.37,1.73,14.31,1.02,21.76,6.06,2007.91,173867.17
std,41.65,22.60,5832.44,1.32,1.08,27.55,20.78,189.73,410.48,186.25,...,121.59,59.17,64.31,17.14,49.66,24.28,223.55,2.58,1.30,69561.87
min,20.00,21.00,1300.00,2.00,3.00,1880.00,1950.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2006.00,35000.00
25%,20.00,56.00,7490.00,5.00,5.00,1953.00,1963.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,2007.00,129000.00
50%,50.00,69.00,9259.00,6.00,5.00,1970.00,1990.00,0.00,404.00,0.00,...,0.00,16.00,0.00,0.00,0.00,0.00,0.00,6.00,2008.00,156000.00
75%,70.00,80.00,11366.50,7.00,6.00,1997.00,2002.00,148.00,695.50,0.00,...,166.00,62.00,0.00,0.00,0.00,0.00,0.00,7.00,2009.00,206500.00
max,190.00,155.00,70761.00,10.00,9.00,2008.00,2009.00,1224.00,1965.00,1526.00,...,646.00,365.00,364.00,219.00,287.00,576.00,4500.00,12.00,2010.00,538000.00


In [4]:
# variable preparation

predictors=list(train.columns[2:-1])
continuous_predictors=train.select_dtypes(['int64']).columns
columns_categorical=train.select_dtypes(['object']).columns


## Step 1: Categorial variables clean and discussion

In [5]:
train['MSZoning'].unique()

array(['RL', 'FV', 'RM', 'RH', 'C (all)', 'I (all)'], dtype=object)

In [6]:
# these variables are part of 'columns_categorical', 
categorical=['MSZoning', 'Street','LotShape', 'LandContour','Utilities','LotConfig', 'LandSlope'
           , 'Neighborhood', 'Condition1', 'Condition2' ,'BldgType','HouseStyle', 'RoofStyle', 'RoofMatl', 
             'Exterior1st','Exterior2nd','ExterQual', 'ExterCond', 'Foundation','Heating'
            , 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual','Functional'
            , 'PavedDrive','SaleType']


train['MSZoning']=train['MSZoning'].str.replace('(',"")
train['MSZoning']=train['MSZoning'].str.replace(')',"")
for item1 in categorical:
    train[item1]=train[item1].str.replace(' ',"")
    length=len(train[item1].unique())
    count=0
    while (length-1)>=count:
        for item2 in train[item1].unique():
            count1=str(count)
            train[item1]=train[item1].str.replace(item2,count1)
            count=int(count1)+1

In [7]:
#This is to replace NaN with empty string for now
train = train.replace(np.nan, '', regex=True)

#Use this to replace whatever first argument is with second argument
#train['Alley'] = train['Alley'].replace(0,'' , regex=True)


### Only 3 values with pool variables, we can just drop this

In [8]:
del train['PoolQC']
del train['PoolArea']
del test['PoolQC']
del test['PoolArea']

In [9]:
df = pd.DataFrame({'Fence':list(train['Fence'])})
pd.get_dummies(df).head()
#Here, the base case Fence means the house does not have a 

,Fence_,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


23 places have a feature out of 805 observations. Is it worthwhile to even include? Only 2% of observations have this

Therefore, I decided to remove it.

In [10]:
del train['MiscFeature']
del test['MiscFeature']

In [11]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,0,,6853,0,,0,0,0,0,...,25,0,0,0,,0,6,2009,0,220000
791,160,1,30,3180,0,Pave,1,0,0,0,...,166,0,0,0,,0,6,2006,0,151000
571,20,0,60,6960,0,,1,0,0,0,...,0,0,0,0,,500,11,2009,0,120500
224,20,0,70,8050,0,,1,0,0,0,...,64,0,0,0,,0,3,2007,0,127000
498,20,0,,9556,0,,0,0,0,1,...,21,0,0,0,,0,8,2007,0,160000


In [12]:
del train['Alley']
del test['Alley']

In [13]:
#Need to conver any string into integer for machine learning algorithm to work
combined_set = pd.concat([train, test], axis = 0)

In [14]:
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

In [15]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 8 to 802
Data columns (total 75 columns):
MSSubClass       804 non-null int64
MSZoning         804 non-null int8
LotFrontage      804 non-null int8
LotArea          804 non-null int64
Street           804 non-null int8
LotShape         804 non-null int8
LandContour      804 non-null int8
Utilities        804 non-null int8
LotConfig        804 non-null int8
LandSlope        804 non-null int8
Neighborhood     804 non-null int8
Condition1       804 non-null int8
Condition2       804 non-null int8
BldgType         804 non-null int8
HouseStyle       804 non-null int8
OverallQual      804 non-null int64
OverallCond      804 non-null int64
YearBuilt        804 non-null int64
YearRemod/Add    804 non-null int64
RoofStyle        804 non-null int8
RoofMatl         804 non-null int8
Exterior1st      804 non-null int8
Exterior2nd      804 non-null int8
MasVnrType       804 non-null int8
MasVnrArea       804 non-null int16
ExterQual    

In [16]:
final_train = combined_set[:train.shape[0]] # Up to the last initial training set row
final_test = combined_set[train.shape[0]:] # Past the last initial training set row

In [17]:
#This is the price variable we need to remove
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')

In [18]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

ImportError: No module named keras.models

# Time for some Adaptive boosting with Decision Trees

In [19]:
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

### Testing to see if I can actually use Adaboost

In [20]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html

regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=9),random_state=0)
regr.fit(final_train,y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=50, random_state=0)

In [21]:
from sklearn.model_selection import cross_val_score
#This uses cross validation to evaluate the score
cross_val_score(regr, final_train, y_train, cv=10)

array([ 0.92690118,  0.81073481,  0.87656596,  0.91526276,  0.88009621,
        0.8243043 ,  0.91016168,  0.85105471,  0.92049621,  0.87521091])

# Bringing it all together to find optimal level of nodes to use!

In [41]:
values = range(1,20)
maximum = 0
optimal_node = 0
for i in values:
    regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=i),random_state=0)
    print("Cross val score for {} nodes".format(i))
    print(cross_val_score(regr, final_train, y_train, cv=10))
    tot = 0
    #Check to see if this layer is better since highest cross_val_score
    for x in g:
        tot = tot+x
    if tot>maximum:
        optimal_node = i
       

Cross val score for 1 nodes
[ 0.43194967 -0.07552333  0.4085615   0.69157937  0.63566335  0.44982422
  0.67708075  0.62614049  0.513767    0.57519268]
Cross val score for 2 nodes
[ 0.81381067  0.62684285  0.72330006  0.81694396  0.80062049  0.75803133
  0.88022217  0.78448873  0.8483366   0.79279161]
Cross val score for 3 nodes
[ 0.87886522  0.72762968  0.84124083  0.87463185  0.8244554   0.79033548
  0.86854456  0.82773734  0.88985325  0.88305757]
Cross val score for 4 nodes
[ 0.91857647  0.77861787  0.87635817  0.87839681  0.86983494  0.83761466
  0.90024462  0.8386563   0.88609915  0.88148635]
Cross val score for 5 nodes
[ 0.91387681  0.82142229  0.87106513  0.8822558   0.89436731  0.81939982
  0.89882463  0.86193278  0.88974658  0.88593843]
Cross val score for 6 nodes
[ 0.90557817  0.82077702  0.90730732  0.9037636   0.88496803  0.82415488
  0.91063483  0.85863682  0.90914347  0.88926782]
Cross val score for 7 nodes
[ 0.91254776  0.82415891  0.8939906   0.91409018  0.88296425  0.81

In [23]:
from sklearn.metrics import r2_score
print("Optimal nodes is {}".format(19))
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=19),random_state=0)
print("Cross val score for {} nodes".format(19))
print(cross_val_score(regr, final_train, y_train, cv=10))

regr.fit(final_train, y_train)
y_1 = regr.predict(final_test)
accuracy = r2_score(y_test, y_1)
print("Cross-Predicted Accuracy: {}".format(accuracy))

Optimal nodes is 19
Cross val score for 19 nodes
[ 0.92433361  0.81603069  0.88490677  0.90816879  0.88445049  0.81368237
  0.90009285  0.87982209  0.90687126  0.88195048]
Cross-Predicted Accuracy: 0.857095403248


In [33]:
from QBUS2820 import rmse_jack, r2_jack 
#Jackknife method to be able to report the standard errors for the test results

columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Results']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, y_1)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, y_1))
results.round(3)

,Test RMSE,SE,Test R2,SE
Results,26844.18,4555.389,0.857,0.031
